In [16]:
import pandas as pd
import numpy as np
df = pd.read_csv('../Data/sms_banking.csv',header=None)

In [17]:
pd.set_option('display.max_colwidth', None)###None: Not limit colwidth

In [18]:
df.columns = ['content']

###tao 1 dataframe chua cac thong tin sau:
Date, Debit(ghi no), Credit (ghi co), Balance (so du)

In [19]:
df

,content
0,"Nhap ma OTP 6190 de xac nhan chuyen 6.170.000 VND cho NGUYEN THI BICH THU,so tai khoan 0560109234007, phi GD 0 VND. Quy khach KHONG CUNG CAP OTP cho bat ky ai."
1,"TK ViettelPay 9704...0765 GD -525,000 VND luc 2021-12-28 19:57:48 So du 1,127,637 VND Phi: -0 VND ND: GD chuyen tien 211228799060765 016537 ck"
2,"TK ViettelPay 9704...0765 GD +1,232,043 VND luc 2021-12-27 13:31:26 So du 1,652,637 VND ND: GD nhan tien 195805"
3,"SD TK 0011004331555 +49,990,000VND luc 20-08-2022 11:02:01. SD 50,568,970VND. Ref 020097042308201102002022d7d1985460.79636.110201.CHUYEN KHOAN"
4,"Ma OTP xac thuc GD la 389235, hieu luc 1 phut. Chi tiet GD: Chuyen khoan nhanh qua so TK, so tien 864,439 VND tren kenh Mobile cua dich vu VCB Digibank."
5,"SD TK 0011004331555 -864,439VND luc 20-08-2022 19:39:54. SD 49,124,531VND. Ref MBVCB.2361079139.085523. Lan chuyen tien.CT tu 001100433155..."
6,"Thu phi thuong nien the Visa Debit, tu thang 06/2022 den thang 06/2022, No 428310...3712. So tien 5000 VND. So du tai thoi diem gd 49,119,531 VND"


In [5]:
import re

In [20]:
df['content']=df['content'].str.replace(',','')

In [21]:
df

,content
0,Nhap ma OTP 6190 de xac nhan chuyen 6.170.000 VND cho NGUYEN THI BICH THUso tai khoan 0560109234007 phi GD 0 VND. Quy khach KHONG CUNG CAP OTP cho bat ky ai.
1,TK ViettelPay 9704...0765 GD -525000 VND luc 2021-12-28 19:57:48 So du 1127637 VND Phi: -0 VND ND: GD chuyen tien 211228799060765 016537 ck
2,TK ViettelPay 9704...0765 GD +1232043 VND luc 2021-12-27 13:31:26 So du 1652637 VND ND: GD nhan tien 195805
3,SD TK 0011004331555 +49990000VND luc 20-08-2022 11:02:01. SD 50568970VND. Ref 020097042308201102002022d7d1985460.79636.110201.CHUYEN KHOAN
4,Ma OTP xac thuc GD la 389235 hieu luc 1 phut. Chi tiet GD: Chuyen khoan nhanh qua so TK so tien 864439 VND tren kenh Mobile cua dich vu VCB Digibank.
5,SD TK 0011004331555 -864439VND luc 20-08-2022 19:39:54. SD 49124531VND. Ref MBVCB.2361079139.085523. Lan chuyen tien.CT tu 001100433155...
6,Thu phi thuong nien the Visa Debit tu thang 06/2022 den thang 06/2022 No 428310...3712. So tien 5000 VND. So du tai thoi diem gd 49119531 VND


In [23]:
###giao dich cong, tru 
pattern = r'TK ViettelPay .+ GD (?P<Trans_Amount>[+|-]\d+) VND luc(?P<Date>(\d|-|\s|:)+) So du (?P<Balance>\d+)'
content_df2 = df['content'].str.extractall(pattern, flags=re.IGNORECASE)
content_df2

,,Trans_Amount,Date,2,Balance
,match,,,,
1,0,-525000,2021-12-28 19:57:48,8,1127637
2,0,+1232043,2021-12-27 13:31:26,6,1652637


In [9]:
###giao dich cong, tru 
pattern = r'SD TK \d+ (?P<Trans_Amount>[+|-]\d+)VND luc (?P<Date>(\d|-|\s|:)+). SD (?P<Balance>\d+)'
content_df1 = df['content'].str.extractall(pattern, flags=re.IGNORECASE)

In [10]:
content_df1

,,Trans_Amount,Date,2,Balance
,match,,,,
3,0,+49990000,20-08-2022 11:02:01,1,50568970
5,0,-864439,20-08-2022 19:39:54,4,49124531


In [12]:
result = pd.concat([content_df1,content_df2],axis=0).drop(2,axis=1)
result

,,Trans_Amount,Date,Balance
,match,,,
3,0,+49990000,20-08-2022 11:02:01,50568970
5,0,-864439,20-08-2022 19:39:54,49124531


In [13]:
result['Trans Type'] = result['Trans_Amount'].apply(lambda x: 'Debit' if '-' in x else 'Credit')
result

,,Trans_Amount,Date,Balance,Trans Type
,match,,,,
3,0,+49990000,20-08-2022 11:02:01,50568970,Credit
5,0,-864439,20-08-2022 19:39:54,49124531,Debit


In [14]:
result['Trans_Amount'] = result['Trans_Amount'].str.replace(r'[+|-]','',regex=True).astype('float')

In [15]:
result

,,Trans_Amount,Date,Balance,Trans Type
,match,,,,
3,0,49990000.0,20-08-2022 11:02:01,50568970,Credit
5,0,864439.0,20-08-2022 19:39:54,49124531,Debit
